In [3]:
%%bash
cat <<EOT > nuxeo.properties
url=http://127.0.0.1:8080/nuxeo
login=$NXUSER
password=$NXPWD
EOT
#cat nuxeo.properties

### States hierarchy

Import the 52 states in 2 differents streams

In [4]:
!import.sh -o consumertree -l import/states-hierarchy -m 

Using config /nxbench/notebooks/11B-Steps/nuxeo.properties
url=http://127.0.0.1:8080/nuxeo
login=nco-admin
Nuxeo Client configured
Connected to Nuxeo Server 11.3.26
Running Operation:StreamImporter.runConsumerFolderProducers
   nbThreads: 10 
   split: true 
   logName: import/states-hierarchy 
   logSize: 8 
#####################
Execution completed
elapsed:0.08
failures:0
messages:52
throughput:650.0
producers:1



### Customers

Import the 89,997,827 Customers/IDCards in 2 differents streams

In [8]:
!csvImport.sh -t 10 -p 16 -o ConsumerProducers -serverThreads 32 -b 50000 \
  -f ../../data/sources/all-id-cards.csv -m -l import/customers > import-customers-csv.log

In [10]:
!tail  import-customers-csv.log

 Tasks count: 1800
 ############################## 
 CSV stats:
    Throughput:42408 lines/s
    Total lines:89997827
 Producers stats:
    total producers:57600
    total messages:179995654
    total failures:0
    Throughput:84796


### Accounts

Import the 179,998,862 Accouts/Correspondence in 2 differents streams

In [11]:
!csvImport.sh -t 10 -p 16 -o AccountProducers -serverThreads 32 -b 50000 \
  -f ../../data/sources/all-accounts.csv -m -l import/accounts > import-accounts-csv.log

In [12]:
!tail import-accounts-csv.log

 Tasks count: 3600
 ############################## 
 CSV stats:
    Throughput:41178 lines/s
    Total lines:179998862
 Producers stats:
    total producers:115200
    total messages:359997724
    total failures:0
    Throughput:82339


### Non Archived Statememnts

Generate 179,998,862*6 statements in 2 different streams.

Because of a bug in the data generation lib, we need to generate with a month offset of 6 to actually get the last 6 months.

In [23]:
ns = 179998862* 6
print("nb statements =", ns)
print("expected duration (s) =", ns/40000)

nb statements = 1079993172
expected duration (s) = 26999.8293


In [ ]:
!import.sh -o statements -l import/statements_live -m  -d 6 -monthOffset 6  \
  -smartPartitioning -n 1079993172 -t 240 -p 24 -a -w 30000 > import-statements-live.log 

In [14]:
!tail  import-statements-live.log 

Async Automation Execution Scheduled
  => status url:[http://127.0.0.1:8080/nuxeo/site/api/v1/automation/StreamImporter.runStatementProducers/@async/20720a6a-502e-40d8-b8f0-a7db8b5e98a0/status]
#####################
Execution completed

waiting for end of Async Exec
url=http://127.0.0.1:8080/nuxeo
login=nco-admin
Nuxeo Client configured
...............................................................................................................................................................................................................................................................................................................................................................................................................

Producers status: threads: 240, failures: 0, messages: 1079993280, elapsed: 27104.99s, throughput: 39844.81 msg/s

### Achived Statements - part 0

This time, we generate in single repository and without offset to get the first 6 months of the year.

In [22]:
ns = 179998862* 6
print("nb statements =", ns)
print("expected duration (s) =", ns/40000)

nb statements = 1079993172
expected duration (s) = 26999.8293


In [15]:
!import.sh -o statements -l import/statements_archive0 -d 6 -monthOffset 0  \
  -smartPartitioning -storeInRoot -n 1079993172 -t 240 -p 24 -a -w 30000 > import-statements-archive0.log

In [16]:
!tail  import-statements-archive0.log

Nuxeo Client configured
..........................................................................................................................................................................................................................................................................................................................................................................................................................................................
Running completed
elapsed:26570.171
failures:0
messages:1079993280
throughput:40646.83211861904
producers:240

Exit after 26585 s


### Achived Statements - part 1

Generate Statements 6 months in single repository mode.

In [18]:
ns = 179998862* 6
print("nb statements =", ns)
print("expected duration (s) =", ns/40000)

nb statements = 1079993172
expected duration (s) = 26999.8293


In [ ]:
!import.sh -o statements -l import/statements_archive1 -d 6 -monthOffset 12  \
  -smartPartitioning -storeInRoot -n 1079993172 -t 240 -p 24 -a -w 65000 > import-statements-archives1a.log

### Archives Statements - part 2
Generate Statements 6 months in single repository mode.

In [ ]:
#!import.sh -o statements -l import/statements_archive2 -d 6 -monthOffset 18  \
  -smartPartitioning -storeInRoot -n 1079993172 -t 240 -p 24 -a -w 65000 > import-statements-archives2.log

### Achived Statements - part 3

1 year

In [ ]:
!import.sh -o statements -l import/statements_archive3 -d 12 -monthOffset 24  \
  -smartPartitioning -storeInRoot -n 2159986344 -t 240 -p 24 -a -w 65000 > import-statements-archives3.log

### Achived Statements - part 4

In [ ]:
!import.sh -o statements -l import/statements_archive4 -d 12 -monthOffset 36  \
  -smartPartitioning -storeInRoot -n 2159986344 -t 240 -p 24 -a -w 65000 > import-statements-archives4.log

### Achived Statements - part 5

In [ ]:
!import.sh -o statements -l import/statements_archive5 -d 12 -monthOffset 48  \
  -smartPartitioning -storeInRoot -n 2159986344 -t 240 -p 24 -a -w 65000 > import-statements-archives5.log